In [11]:
## import libraries
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model


In [12]:
## Load the imdb dataset word index
word_index = imdb.get_word_index()
reversed_word_index = {value:key for key,value in word_index.items()}
list(reversed_word_index.items())[0:10]

[(34701, 'fawn'),
 (52006, 'tsukino'),
 (52007, 'nunnery'),
 (16816, 'sonja'),
 (63951, 'vani'),
 (1408, 'woods'),
 (16115, 'spiders'),
 (2345, 'hanging'),
 (2289, 'woody'),
 (52008, 'trawling')]

In [ ]:
## loadint the pretrained model with ReLu activation.
model = load_model('simple_rnn_imdb.h5')
model.summary()

I0000 00:00:1758312300.773586   13107 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5581 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [14]:
model.get_weights()[0:2]

[array([[ 0.1449298 ,  0.19248587, -0.37181252, ...,  0.05987724,
          0.13583647,  0.26952475],
        [ 0.01192128, -0.02350695, -0.05522561, ...,  0.0372317 ,
          0.00546152,  0.01412222],
        [ 0.16927649,  0.00566336, -0.09862465, ...,  0.11414826,
          0.13341321,  0.01353813],
        ...,
        [-0.08123806,  0.10627506,  0.02234682, ...,  0.0072253 ,
         -0.05036665, -0.0167215 ],
        [-0.12243535,  0.06385783, -0.05546444, ..., -0.11771943,
         -0.03490656, -0.08075718],
        [ 0.19210465, -0.01254928,  0.3107335 , ...,  0.1733949 ,
          0.18177287,  0.25477722]], shape=(10000, 128), dtype=float32),
 array([[ 2.07491681e-01,  2.00836554e-01, -1.74224198e-01, ...,
          1.12244248e-01, -1.11062769e-02, -9.81981793e-05],
        [ 1.49862841e-01,  7.58388117e-02,  1.16683997e-01, ...,
         -3.06964274e-02,  4.53796908e-02, -1.85298715e-02],
        [ 2.23762602e-01,  1.86290592e-01,  9.25832912e-02, ...,
         -1.16972812e

In [15]:
## Function to decode the reviews(converts the numbers into words)
def decode_review(encode_review):
    return " ".join([reversed_word_index.get(i-3,'?') for i in encode_review])

## Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review =[word_index.get(word,2)+3  for word in words]
    padded_review = sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

In [19]:
## Prediction fn
def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)
    sentiment ="positive" if prediction[0][0]>0.5 else "Negative"
    return sentiment,prediction[0][0]

In [20]:
example_review = "This movie was fantastic! The acting was great and the plot was thrilling "
setntiment,prediction_score = predict_sentiment(example_review) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [21]:
print(f"sentiment:{setntiment},prediction_score:{prediction_score}")

sentiment:positive,prediction_score:0.7248525023460388
